In [12]:
import time
from pynq import Overlay
import numpy as np
from pynq import Xlnk as xlnk
import struct
from scipy.misc import imread
import cv2

xlnk=Xlnk();
#input image
image=xlnk.cma_array(shape=(28*28*1),cacheable=0,dtype=np.float32)
#weight
weight_all=xlnk.cma_array(shape=(1*16*3*3+16*32*3*3+32*128*7*7+128*10*1*1),cacheable=0,dtype=np.float32)
#bias
bias_all=xlnk.cma_array(shape=(16+32+128+10),cacheable=0,dtype=np.float32)
#out
result=xlnk.cma_array(shape=(10),cacheable=0,dtype=np.float32)

In [13]:
idx = 0

w = np.fromfile("W_conv1.bin", dtype=np.float32)
np.copyto(weight_all[idx:idx+len(w)], w)
idx +=len(w)

w = np.fromfile("W_conv2.bin", dtype=np.float32)
np.copyto(weight_all[idx:idx+len(w)], w)
idx +=len(w)

w = np.fromfile("W_fc1.bin", dtype=np.float32)
np.copyto(weight_all[idx:idx+len(w)], w)
idx +=len(w)

w = np.fromfile("W_fc2.bin", dtype=np.float32)
np.copyto(weight_all[idx:idx+len(w)], w)
idx +=len(w)

idx = 0
b = np.fromfile("b_conv1.bin", dtype=np.float32)
np.copyto(bias_all[idx:idx+len(b)], b)
idx +=len(b)

b = np.fromfile("b_conv2.bin", dtype=np.float32)
np.copyto(bias_all[idx:idx+len(b)], b)
idx +=len(b)

b = np.fromfile("b_fc1.bin", dtype=np.float32)
np.copyto(bias_all[idx:idx+len(b)], b)
idx +=len(b)

b = np.fromfile("b_fc2.bin", dtype=np.float32)
np.copyto(bias_all[idx:idx+len(b)], b)
idx +=len(b)

In [14]:
image1=cv2.imread("4.jpg",cv2.IMREAD_GRAYSCALE).astype(np.float32)
print("Read image")

for i in range(28):
    for j in range(28):
        for k in range(1):
            image[i*28*1+j*1+k]=(255-image1[i][j])/255

Read image


In [18]:
overlay = Overlay("./lenet.bit")
print("Bitstream loaded")
overlay?

/usr/local/lib/python3.6/dist-packages/pynq/pl_server/device.py:594: UserWarning: Users will not get PARAMETERS / REGISTERS information through TCL files. HWH file is recommended.
  warnings.warn(message, UserWarning)


Bitstream loaded


In [19]:
Lenet =overlay.lenet_0
Lenet.write(0x10, image.physical_address)
Lenet.write(0x18, weight_all.physical_address)
Lenet.write(0x20, bias_all.physical_address)
Lenet.write(0x28, result.physical_address)

In [20]:
Lenet.write(0x00, 1)
isready = Lenet.read(0x00)
while( isready == 1 ):
    isready = Lenet.read(0x00)
print(result)
MAX = 0
num = 0
for i in range(0,10):
    if(result[i]>MAX):
        MAX=result[i]
        num=i
print("The number you write is "+str(num))

[  0.           0.           0.           0.          12.53744698   0.           0.
   0.           0.           3.6278832 ]
The number you write is 4
